# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [15]:
# import libraries
from sqlalchemy import create_engine

import pandas as pd 
import numpy as np
import re
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn.multiclass import OneVsRestClassifier

import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /Users/macbook/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/macbook/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
# load data from database
engine = create_engine('sqlite:///IDisasterResponse.db')
df = pd.read_sql_table("DisasterResponse", con=engine)

X = df["message"]
Y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)

### 2. Write a tokenization function to process your text data

In [17]:
def tokenize(text):
    
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:

pipeline  = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

In [20]:
# predict on test data
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:

#target_names = Y.columns
def model_testing(y_test, y_pred):
    for i in range(36):
        print("classification report for " + y_test.columns[i]
              ,'\n', classification_report(y_test.values[:,i],y_pred[:,i])
              ,'\n accuracy:', accuracy_score(y_test.values[:, i], y_pred[:, i]))

        
model_testing(y_test, y_pred)

classification report for related 
               precision    recall  f1-score   support

           0       0.74      0.29      0.42      1518
           1       0.82      0.97      0.89      5036

    accuracy                           0.81      6554
   macro avg       0.78      0.63      0.65      6554
weighted avg       0.80      0.81      0.78      6554
 
 accuracy: 0.8121757705218188
classification report for request 
               precision    recall  f1-score   support

           0       0.89      0.99      0.94      5456
           1       0.89      0.42      0.57      1098

    accuracy                           0.89      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554
 
 accuracy: 0.8941104668904486
classification report for offer 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy  

/Users/macbook/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


classification report for buildings 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      6222
           1       0.82      0.07      0.13       332

    accuracy                           0.95      6554
   macro avg       0.89      0.53      0.55      6554
weighted avg       0.95      0.95      0.93      6554
 
 accuracy: 0.9520903265181568
classification report for electricity 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6411
           1       0.67      0.06      0.10       143

    accuracy                           0.98      6554
   macro avg       0.82      0.53      0.55      6554
weighted avg       0.97      0.98      0.97      6554
 
 accuracy: 0.9787915776624961
classification report for tools 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6511
           1       0.00      0.00      0.00        43

    accu

### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__min_samples_split': [2, 4]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=4)
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed: 43.3min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
y_pred = cv.predict(X_test)
model_testing(y_test, y_pred)
print("\nBest Parameters:", cv.best_params_)

classification report for related 
               precision    recall  f1-score   support

           0       0.74      0.28      0.41      1518
           1       0.82      0.97      0.89      5036

    accuracy                           0.81      6554
   macro avg       0.78      0.63      0.65      6554
weighted avg       0.80      0.81      0.78      6554
 
 accuracy: 0.8101922490082393
classification report for request 
               precision    recall  f1-score   support

           0       0.90      0.99      0.94      5456
           1       0.86      0.44      0.58      1098

    accuracy                           0.89      6554
   macro avg       0.88      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554
 
 accuracy: 0.893500152578578
classification report for offer 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy   

/Users/macbook/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


classification report for hospitals 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6477
           1       0.00      0.00      0.00        77

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554
 
 accuracy: 0.9882514494964907
classification report for shops 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554
 
 accuracy: 0.995575221238938
classification report for aid_centers 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6470
           1       0.00      0.00      0.00        84

    accur

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [24]:
svm_pipeline  = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state=0))))
    ])
svm_pipeline.fit(X_train, y_train)
y_pred = svm_pipeline.predict(X_test)
model_testing(y_test, y_pred)

/Users/macbook/opt/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


classification report for related 
               precision    recall  f1-score   support

           0       0.66      0.53      0.59      1518
           1       0.87      0.92      0.89      5036

    accuracy                           0.83      6554
   macro avg       0.77      0.73      0.74      6554
weighted avg       0.82      0.83      0.82      6554
 
 accuracy: 0.8291119926762283
classification report for request 
               precision    recall  f1-score   support

           0       0.92      0.96      0.94      5456
           1       0.77      0.59      0.67      1098

    accuracy                           0.90      6554
   macro avg       0.85      0.78      0.81      6554
weighted avg       0.90      0.90      0.90      6554
 
 accuracy: 0.9020445529447666
classification report for offer 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy  

/Users/macbook/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


classification report for hospitals 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6477
           1       0.20      0.01      0.02        77

    accuracy                           0.99      6554
   macro avg       0.59      0.51      0.51      6554
weighted avg       0.98      0.99      0.98      6554
 
 accuracy: 0.9877937137625877
classification report for shops 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554
 
 accuracy: 0.995575221238938
classification report for aid_centers 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6470
           1       0.50      0.01      0.02        84

    accur

### 9. Export your model as a pickle file

In [25]:

with open('svm_pipeline_model.pkl', 'wb') as file:
    pickle.dump(svm_pipeline, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.